In [ ]:
# header / imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sc3nb as scn
import time

In [ ]:
sc = scn.startup()

# Example for sc3nb.SynthDef and sc3nb.Synth

## sc3nb.SynthDef

SynthDef wraps and extends the flexibility of SuperCollider Synth Definitions
* SynthDefs are created via sclang and send to the server via the sclangs .add() method
* the sc3nb synthdef class - different from standard sclang synth defs - allows to inject a number of context specifiers using mustache syntax, i.e. {{my_context_specifier}} would mark a place within the synth definition that can later be replaced by some use-wished sclang code.

You can create a pseudo synth definition with the SynthDef subclass in sc:

In [ ]:
synthDef = sc.SynthDef(name="myKlank", definition=r"""
{ |out=0, amp=0.3, freq=440|
    var klank = DynKlank.ar(`[[1,2], [1,1], [1.4,1]], {{EXCITER}}, freq);
    Out.ar(out, amp*klank!^p_channels);
}""")

In this example a new definition named "myKlank" offers a dynamic context "EXCITER"
and 3 value-contexts freqs, rings, and channels.
Now we want to replace the dynamic context EXCITER with a specific code.
* use set_context to replace a context with specific code
* the specific code can include other context specifier strings in turn
    * this basically allows to create chained processing in python-compiled synths defs.

In [ ]:
synthDef.set_context("EXCITER", "Dust.ar(20)")

**Remarks**:
* The context mechanism is very general:
    * it can be used both for code and values
    * e.g. for an array specification or a UGen selection.
* To set a value (e.g. number or array), consider to use the pyvars syntax, i.e. using the caret '^' variable value injection of python variables into sc code

The definition is complete up to the pyvars. So let's create a myKlank in SC!

* **Note that in this case pyvars need to be filled in... Why?**

In [ ]:
kdust = synthDef.reset().set_context("EXCITER", "Dust.ar(80)").add(
    pyvars={"p_channels": 2})
print(kdust)

In [ ]:
# for testing, let's create a synth and stop after 1s
%scv x = Synth.new(^kdust, [\freq, 200, \amp, 0.05])
time.sleep(1)
%scv x.free

Now let's create another synthdef with a WhiteNoise excitation, but as a 1-channel version

In [ ]:
p_channels = 1

In [ ]:
knoise = synthDef.reset().set_context("EXCITER", "WhiteNoise.ar(0.2)").add()
knoise

In [ ]:
%scv s.freeAll

In [ ]:
# for testing, let's create a synth and stop after 1s
%scv x = Synth.new(^knoise, [\amp, 0.05, \freq, 100])
time.sleep(1)
%scv x.free

To delete an existing synthdef in sc 
you can use

        synthDef.free(name)

(but don't do it now as the synthdef is used for examples below)

Remove all unused placeholders from current_def by

In [ ]:
synthDef.reset()
print(f"SC code with context vars: \n {synthDef.current_def} \n")
synthDef.unset_remaining()
print(f"SC code with unset context vars: \n {synthDef.current_def} \n", )

Here you see, that the placeholder {{EXCITER}} has been deleted. 
* With this method you make sure, that you don't have any unused placeholders in the definition before creating a SynthDef.
* Note, however, that your code might then not be functional...

## sc3nb.Synth

To use the above new created synthDef (or another one you'd already have)..

In [ ]:
synthDef = sc.SynthDef(name="myKlank", definition=r"""
{ |out=0, amp=0.05, freqs=#[1,2,3], rings=#[1,2,3], freq=100 |
    Out.ar(out, amp * DynKlank.ar(`[freqs, nil, rings], {{EXCITER}}, freq)!2
    )
}""")

In [ ]:
kimpulse = synthDef.reset().set_context("EXCITER", "Impulse.ar(10)").add()
print(kimpulse)

Create a new synth:

In [ ]:
synth = sc.Synth(name=kimpulse, args={"freq": 100})

In [ ]:
synth.free()

Start the synth again

In [ ]:
synth.start()

Pause a synth

In [ ]:
synth.pause()

Run a paused synth:

In [ ]:
synth.run()

Set synth parameters, using any for the following calls:

        set(key, value, ...)
        set(list_of_keys_and_values])
        set(dict)

In [ ]:
synth.set({"freq": 130, "amp": 0.01})

In [ ]:
synth.set(["freq", 30, "amp", 0.02])

In [ ]:
synth.set("amp", 0.01)

You can use ``get`` to see the current value 

In [ ]:
synth.get("rings")

You can also see what arguments can be set with the synth_args attribute

In [ ]:
synth.synth_args

And check thier default argument

In [ ]:
synth.synth_args['freqs'].default

These can be accessed to see what is the current value and also to set a new value.

In [ ]:
synth.freq

In [ ]:
synth.freq = 60.0

In [ ]:
synth.rings

In [ ]:
synth.rings = synth.rings[::-1]

In [ ]:
synth.rings

Free a running synth:

In [ ]:
synth.free()

You can also use the `return_msg` flag to get OscMessages and send them as a bundle

In [ ]:
now = time.time()
bundle = sc.bundle(now)
bundle.add(sc.bundle(now + 1.0).add(synth.start(return_msg=True)).build())
bundle.add(sc.bundle(now + 2.0).add(synth.set(['freq', synth.current_args['freq'] * 2], return_msg=True)).build())
bundle.add(sc.bundle(now + 3.0).add(synth.free(return_msg=True)).build())
bundle.send()

Refer to the osc communication example notebook if you want to learn more about messages and bundles.

## Example creation of many SynthDefs
In some cases you want to create many SynthDefs with only a small change. You can use the SynthDefs object multiple time to do this. Here we want to create playbuf synthdefs for 1 to 10 channels:
(Reuse of the synthdef object, which is defined above)

In [ ]:
synthPlaybufs = {}
for channel in [1,2,4,8]:
    synthPlaybufs[channel] = synthDef.add(pyvars={"channel": channel})

Now you can access via ``synthPlayBufs[2]`` to the 2-ch playbuf etc.

In [ ]:
synthPlaybufs[2]

## Use-case: DynKlank Synths with controllable nr. of filters
A problem with synthdefs is that some parameters can only be set at compile time. E.g. 
* A DynKlank needs to know the max nr. of filters in its filter bank at SynthDef time. 
* A synth will need to know the channel count at synthdef time

Contexts allow to define such synthDefs dynamically on demand. 

The following code is a dynamic DynKlank whose data-controlled nr. of filters is determined via the SynthDef class. 
* channel number and number of filters in filter bank is defined via py_vars
* TODO: find a way how to set amps, rings, and harms on Synth.new

In [ ]:
synthDef = sc.SynthDef(name="myKlank", definition=r"""
{ |out=0, amp=0.3, freq=440|
    var klank, n, harms, amps, rings;
    harms = \harms.kr(Array.series(^p_nf, 1, 1));
    amps = \amps.kr(Array.fill(^p_nf, 0));
    rings = \rings.kr(Array.fill(^p_nf, 0.1));
    klank = DynKlank.ar(`[harms, amps, rings], {{EXCITER}}, freq);
    Out.ar(out, amp*klank!^p_channels);
}""")

# now create a synth where exciter is Dust, with 10 filters and stereo
kdust = synthDef.reset().set_context("EXCITER", "Dust.ar(80)").add(
    pyvars={"p_nf": 10, "p_channels": 2})
print(kdust)

x = sc.Synth(name=kdust, args={"freq": 100, "amp": 0.05})
x.set("harms", [1,2,6], "amps", [0.1,0.1,0.1], "rings", [1, 0.4, 0.2])
# following syntax works the same:
#x.set({"harms":[1,2,6], "amps": [0.1,0.1,0.1], "rings": [1, 0.4, 0.2]})
#x.set(["harms",[1,2,6],"amps",[0.1,0.1,0.1],"rings",[1, 0.4, 0.2]])

time.sleep(2)
x.free()